In [1]:
%load_ext Cython

In [13]:
%%cython
# distutils: language = c++
from __future__ import print_function
import numpy as np
cimport numpy as np
cimport cython
from libcpp.algorithm cimport sort,unique
from libcpp.vector cimport vector
from libc.math cimport ceil
from libc.math cimport pow as pow_C 

from tqdm import tqdm
from joblib import Parallel,delayed
#import multiprocessing


@cython.boundscheck(False) 
@cython.wraparound(False)  
cdef inline double my_abs(double x) nogil:
    if x > 0:
        return x
    else:
        return -x
    
@cython.boundscheck(False) 
@cython.wraparound(False)  
cdef inline double my_dot(double[:] x, double[:] y) nogil:
    cdef int i
    cdef int n = x.shape[0]
    cdef double dot_product = 0
    for i in range(n):
        dot_product += x[i]*y[i]
    return dot_product
        
@cython.boundscheck(False) 
@cython.wraparound(False)    
cdef void my_unique_sorted(vector[double]& x) nogil:
    cdef int n = x.size()
    cdef int i
    for i in range(n-1):
        if x[i+1] != x[i]:
            x.erase(x.begin()+i)

# cdef vector[double] a
# a.reserve(9)
# cdef int i
# for i in range(7):
#     if i == 5:
#         a.push_back(5)
#     a.push_back(float(i))
# print(a)
# print(my_unique_sorted(a))
  

       
# @cython.boundscheck(False) 
# @cython.wraparound(False)  
# cdef double[:] _sort_cpp(double[:] a):
#     # a must be c continuous (enforced with [::1])
#     sort(&a[0], (&a[0]) + a.shape[0])
#     return a 

@cython.boundscheck(False) 
@cython.wraparound(False)    
cdef inline double _Wp(vector[double] P, vector[double] Q, int p = 2) nogil:
    cdef int N1_int = P.size()
    cdef int N2_int = Q.size()
    #cdef int N1_int = P.shape[0]
    #cdef int N2_int = Q.shape[0]
    cdef double N1 = N1_int
    cdef double N2 = N2_int 
    cdef vector[double] U1
    
    # cdef double[:] P_sorted = _sort_cpp(P)
    # cdef double[:] Q_sorted = _sort_cpp(Q)
    sort(P.begin(),P.end())
    sort(Q.begin(),Q.end())
    
    # cdef vector[double] P_sorted
    # P_sorted.reserve(N1_int)
    # for j in range(N1_int):
    #     P_sorted.push_back(P[j])
    # 
    # cdef vector[double] Q_sorted
    # Q_sorted.reserve(N2_int)
    # for j in range(N2_int):
    #     Q_sorted.push_back(Q[j])
    # 
    # sort(P_sorted.begin(),P_sorted.end())
    # sort(Q_sorted.begin(),Q_sorted.end())
    
    U1.reserve(N1_int+1)
    cdef double _i = 1
    U1.push_back(0)
    cdef int j
    cdef int i
    for j in range(N1_int):
        U1.push_back(_i/N1)
        _i += 1.
        
    _i = 1    
    
    cdef vector[double] U2
    U2.reserve(N2_int)
    for j in range(N2_int):
        U2.push_back(_i/N2)
        _i += 1
    cdef vector[double] U
    # concatenate & remove duplicates & sort
    U.reserve(U1.size()+U2.size())
    U.insert( U.end(), U1.begin(), U1.end() )
    U.insert( U.end(), U2.begin(), U2.end() )
    sort(U.begin(),U.end())
    #U.erase(unique(U.begin(),U.end()),U.end())
    my_unique_sorted(U)
    
    
    cdef int N_max = U.size() - 1
    
    #cdef double N_max_double = N_max 
    cdef double dist = 0
    # cdef int index1
    # cdef int index2
    # print(U)
    # print(P)
    # print(Q)
    for i in range(N_max):
        # index1 = int(ceil(U[i]*(N1-1)))
        # index2 = int(ceil(U[i]*(N2-1)))
        dist += pow_C(my_abs(P[int(ceil(U[i]*(N1-1)))] - Q[int(ceil(U[i]*(N2-1)))]),p)*(U[i+1] - U[i])
        #Wp_dist += P[int(ceil(U[i]*(N1-1)))] - Q[int(ceil(U[i]*(N2-1)))]
        #dist += 1
        #print(int(ceil(U[i]*(N1-1))))
    #return pow_C(dist,1./float(p)) 
    return dist

# calculate Wp difference for a subsample 
@cython.boundscheck(False) 
@cython.wraparound(False)    
cdef inline double _calculate_Wp_difference(int[:] A,
                                     int mtry,
                                     double[:] split_direction,
                                     double split_value,
                                     double[:,:] X,
                                     double[:] Y,
                                     int p = 2):
    cdef int A_size = A.shape[0]
    cdef vector[double] Y_left
    cdef vector[double] Y_right
    Y_left.reserve(A_size)
    Y_right.reserve(A_size)
    
    cdef int i
    cdef int _i
    for i in range(A_size):
        _i = A[i]
        if my_dot(X[_i],split_direction) < split_value:
            Y_left.push_back(Y[_i])
            #Y_left.push_back(1)
        else:
            Y_right.push_back(Y[_i])
            #Y_right.push_back(1)
    return _Wp(P = Y_left,Q = Y_right, p = p) 
    #return ot.wasserstein_1d(np.array(Y_left),np.array(Y_right), p = p)

@cython.boundscheck(False) 
#@cython.wraparound(False)    
def getALAR(int[:] A,
            double[:] split_direction,
            double split_value,
            double[:,:] X):
    
    cdef vector[int] _AL
    cdef vector[int] _AR
    cdef int A_size = A.shape[0]
    _AL.reserve(A_size)
    _AR.reserve(A_size)
    for i in range(A_size):
        if my_dot(X[A[i]],split_direction) < split_value:
            _AL.push_back(A[i])
        else:
            _AR.push_back(A[i])
            
    #cdef int[:] AL = _AL.data()
    #cdef int[:] AR = _AR.data()
    
    # cdef int *AL = &_AL[0]    
    # cdef int[::1] AL_view = <int[:_AL.size()]>AL    # cast to typed memory view
    
    # cdef int *AR = &_AL[0]    
    # cdef int[::1] AR_view = <int[:_AL.size()]>AR
    # # 
    return np.asarray(_AL,dtype=np.intc),np.asarray(_AR,dtype=np.intc)
    #return 0 



@cython.boundscheck(False) 
@cython.wraparound(False)    
cdef inline double[:] getDirection(int d):
    cdef double[:] x = np.random.normal(0,1,d)
    cdef int i
    cdef double NUM =0  
    for i in range(d):
        NUM += pow_C(x[i],2)
    NUM = pow_C(NUM,0.5)
    for i in range(d):
        x[i] /= NUM
    return x
        

# Wp_split
@cython.boundscheck(False) 
#@cython.wraparound(False)    
def Wp_split(int[:] A,
             int mtry,
             double[:,:] X,
             double[:] Y,
             int p = 2):
    
    cdef int A_size = A.shape[0]  
    
    cdef int _size_list = (A_size-1)*mtry
    cdef int d = X.shape[1]
    
    cdef double[:,:] direction_list = np.zeros((mtry,d)) 
    #direction_list.reserve(_size_list)
    cdef vector[int] index_direction_list
    cdef vector[double] value_list 
    value_list.reserve(_size_list)
    index_direction_list.reserve(_size_list)
    
    cdef double[:] split_direction = np.zeros(d)
    
    cdef vector[double] Xtry 
    Xtry.reserve(A_size)
    
    cdef int i
    cdef int j
    
    for i in range(mtry):
        split_direction = getDirection(d) 
        #Xtry = np.sort(list(set(X[A,:][:,split_direction])))
        for j in range(A_size):
            Xtry.push_back(my_dot(X[A[j]],split_direction))
        sort(Xtry.begin(),Xtry.end())
        #print("X_before",Xtry.size())
        Xtry.erase(unique(Xtry.begin(),Xtry.end()),Xtry.end())
        #print("X_after",Xtry.size())
        #Xtry = np.sort(list(set(X[A,split_direction])))
        #candidate_current_direction = (Xtry[min_sample_each_node:] + Xtry[:-min_sample_each_node])*.5
        
        direction_list[i] = split_direction
        
        for j in range(Xtry.size() - 1):
            index_direction_list.push_back(i)
            value_list.push_back((Xtry[j+1]+Xtry[j])*0.5)
        Xtry.clear()
        #theta_list += [[split_direction,candidate_current_direction[i]] for i in range(len(Xtry) -min_sample_each_node)]
    
    #print("values:",value_list)
    cdef vector[double] Wp_list
    Wp_list.reserve(value_list.size())
    for i in range(value_list.size()):
        Wp_list.push_back(_calculate_Wp_difference(A = A,
                                                   mtry = mtry,
                                                   split_direction = direction_list[index_direction_list[i]],
                                                   split_value = value_list[i],
                                                   X = X,
                                                   Y = Y,
                                                   p = p))
        
    #cdef int argMax = max_element(Wp_list.begin(), Wp_list.end())-Wp_list.begin()
    # cdef double *Wp_array = &Wp_list[0]    
    # cdef double[::1] Wp_view = <double[:Wp_list.size()]>Wp_array
    #Wp_np = np.asarray(Wp_view)
    
    cdef int index_max = my_argmax(Wp_list) 
# 
    return direction_list[index_direction_list[index_max]], value_list[index_max] 

@cython.boundscheck(False) 
#@cython.wraparound(False)    
cdef inline int my_argmax(vector[double] x) nogil:
    cdef int i
    cdef int _i = 0
    cdef double _x = x[0]
    for i in range(x.size()):
        if x[i] >= _x:
            _i = i
            _x = x[i]
    return _i

In [14]:
class node:
    def __init__(self, left = None, right = None, parent = None, direction = None,value = None, neighbours = None, nodes = None):
        self.left = left 
        self.right = right 
        self.direction = direction  
        self.value = value  
        self.neighbours = neighbours 
        
class DecisionTree:
    def __init__(self,
                 mtry = 1,
                 nodesize = 5,
                 subsample = 100,
                 bootstrap = True,
                 p = 2,
                 nodes = None):
        #self.nodes = nodes # P 
        # parameters
        self.mtry = mtry
        self.nodesize = nodesize
        self.subsample = subsample 
        self.bootstrap = bootstrap 
        self.p = p 
        self.Y = None
        self.root = None
        self.P = None
    def fit(self,X,Y):
        self.Y = Y
        N,d = X.shape 
        # if self.subsample <= 1:
        #     subsample_size = int(N*self.subsample)
        # elif self.subsample > 1:
        #     subsample_size = int(self.subsample)
        # else:
        #     subsample_size = 100
        #     print("Wrong subsample is given, used subsample = 100 as default.")
        S_b = np.random.choice(range(N), self.subsample, replace  = self.bootstrap)
        self.root = node(neighbours = S_b)
        self.P =[self.root] 
        while self.P:
            # A is current node
            A = self.P[0] 
            if len(A.neighbours) < self.nodesize or len(set(Y[A.neighbours])) == 1:
                del self.P[0]
            else:
                #Mtry = np.random.choice(range(d),self.mtry,replace = False)
                #print(type(np.array(Mtry,dtype = int)[0]))
                direction,value = Wp_split(np.array(A.neighbours,dtype = np.intc),np.intc(self.mtry),X,Y,self.p)
                A.direction = direction 
                A.value = value 
                #theta_star_list += [theta_star] 
                AL,AR = getALAR(np.array(A.neighbours,dtype = np.intc),direction,value,X)
                del self.P[0]
                A.left = node(neighbours = AL)
                A.right = node(neighbours = AR)
                self.P += [A.left,A.right]
        
    def _predict(self,x):
        current_node = self.root
        while current_node.value:
            direction = current_node.direction
            value = current_node.value
            if np.dot(x,direction)<value:
                current_node = current_node.left
            else:
                current_node = current_node.right
        return np.mean(self.Y[current_node.neighbours])
    def predict_distribution(self,x):
        current_node = self.root
        while current_node.value:
            direction= current_node.direction
            value= current_node.value
            if np.dot(x,direction)<value:
                current_node = current_node.left
            else:
                current_node = current_node.right
        N_final = self.Y.shape[0]
        empirical_measure = np.zeros(N_final)
        for i in current_node.neighbours:
            empirical_measure[i] += 1.
        return empirical_measure/float(len(current_node.neighbours))
            
    def predict(self,x):
        return np.apply_along_axis(lambda x : self._predict(x),1,x)
        
class WassersteinRandomForest:
    def __init__(self,
                 mtry = 1,
                 nodesize = 5,
                 subsample = 0.1,
                 bootstrap = False,
                 n_estimators = 10,
                 n_jobs = 1,
                 progressbar = True,
                 p = 2):
        # parameters
        self.mtry = mtry
        self.nodesize = nodesize
        self.subsample = subsample 
        self.bootstrap = bootstrap 
        self.n_estimators = n_estimators
        self.n_jobs = n_jobs
        self.progressbar = progressbar
        #self.n_jobs = 1 
        self.p = p
        #self.ListLearners = None 
        self.ListLearners = []
        self.Y = None
        
    def reset_random_state(self):
        f = open("/dev/random","rb")
        rnd_str = f.read(4)
        rnd_int = int.from_bytes(rnd_str, byteorder = 'big')
        np.random.seed(rnd_int)

    def _fit(self,X,Y):
        if self.n_jobs > 1:
            self.reset_random_state()
        BaseLearner = DecisionTree(mtry = self.mtry,
                                   nodesize = self.nodesize,
                                   subsample = self.subsample,
                                   bootstrap = self.bootstrap,
                                   p = self.p)
        BaseLearner.fit(X,Y)
        return BaseLearner

    def fit(self,X,Y):
        self.Y = Y
        if self.n_jobs ==1:
            #self.ListLearners = []
            if self.progressbar:
                fit_tqdm = tqdm(range(self.n_estimators)) 
                for i in fit_tqdm:
                    self.ListLearners += [self._fit(X,Y)]
            else:
                for i in range(self.n_estimators):
                    self.ListLearners += [self._fit(X,Y)]
        else:
            ListLearners = Parallel(n_jobs=self.n_jobs)(delayed(self._fit)(X,Y) for i in tqdm(range(self.n_estimators)))

            #results =\
            #Parallel(n_jobs=self.n_jobs)(delayed(self._fit) (X=X,Y=Y) for i in tqdm(range(self.n_estimators)))
        #fit_tqdm.reset()
            #self.ListLearners = results 
    # def fit(self,X,Y):
    #     self.Y = Y
    #     for i in tqdm(range(self.n_estimators)):
    #         #BaseLearner = DecisionTree(mtry = self.mtry,
    #         #                           nodesize = self.nodesize,
    #         #                           subsample = self.subsample,
    #         #                           bootstrap = self.bootstrap,
    #         #                           p = self.p)
    #         #BaseLearner.fit(X,Y)
    #         self.ListLearners += [self._fit(X,Y)]
    def predict(self,x):
        prediction = np.zeros(x.shape[0])
        for i in range(self.n_estimators):
            prediction += self.ListLearners[i].predict(x)
        prediction /= float(self.n_estimators)
        return prediction
    def predict_distribution(self,x):
        empirical_measure = np.zeros(len(self.Y))
        for i in range(self.n_estimators):
            current_empirical_measure= self.ListLearners[i].predict_distribution(x)
            empirical_measure += current_empirical_measure
        return self.Y,empirical_measure/float(self.n_estimators)

# Generating synthetic dataset 

In [15]:
import matplotlib.pyplot as plt
import numpy as np

N_total =55000
N_train =50000
X = np.random.uniform(0,1,(N_total,5))
#X = np.random.normal(0,1,(N_total,10))
def obj_func(x):
    """
    conditional expectation
    """
    # x0_tilde = 2.*(x[0] - 0.5)
    # x1_tilde = 2.*(x[1] - 0.5)
    # return x0_tilde**2 + x[3]*x[4]
    # c = 0
    # for i in range(3):
    #     #c *= x[i]*np.sin(i)
    #     c += x[i]*np.sin(i)
    return 10*x[1] + x[2]
def obj_func2(x):
    """
    conditional variance
    """
    #return np.abs(np.sin(x[0])+x[1])*1.5
    #return np.abs(x[0]+x[1])*1.5
    return np.max([x[3]*9., 0.2])
    #return 0.5

# def obj_func3(x):
#     """
#     conditional expectation
#     """
#     #return x[1]+2.*x[2] +2. +np.sin(2.*x[0])
#     c = 0
#     for i in range(4,7):
#         #c *= x[i]*np.sin(i)
#         c += x[i]*np.cos(i)
#     #return x[1] + x[2]
#     return c


#Y = np.random.normal(0,1.,N_total) + np.apply_along_axis(obj_func,1,X)
Y = np.zeros(N_total)
for i in range(N_total):
    if np.random.rand()<0.5:
        Y[i] = np.random.normal(obj_func(X[i]),np.sqrt(obj_func2(X[i])),1)
    else:
        #Y[i] = np.random.normal(obj_func(X[i]),np.sqrt(obj_func2(X[i])),1)
        # Y[i] = np.random.normal(-0.05*obj_func2(X[i]),1,1)
        #Y[i] = np.random.normal(-1.5*obj_func2(X[i]),1,1)
        #Y[i] = np.random.normal(obj_func3(X[i]),1,1)

        #Y[i] = np.random.normal(-1,1,1)
        Y[i] = np.random.normal(obj_func(X[i]),np.sqrt(obj_func2(X[i])),1)

In [16]:
N_total = 7000 
N_train = 5000 
X = np.random.uniform(0,1,(N_total,50))
#X = np.random.normal(0,1,(N_total,10))
def obj_func(x):
    """
    conditional expectation
    """
    # x0_tilde = 2.*(x[0] - 0.5)
    # x1_tilde = 2.*(x[1] - 0.5)
    # return x0_tilde**2 + x[3]*x[4] 
    # c = 0
    # for i in range(3):
    #     #c *= x[i]*np.sin(i)
    #     c += x[i]*np.sin(i)
    #return 10.*x[1] + x[2]
    return 10.*x[1]*x[3]**2 + x[2] + np.exp(x[3]-2*x[0]+np.sin(x[1]**3))
def obj_func2(x):
    """
    conditional variance
    """
    #return np.abs(np.sin(x[0])+x[1])*1.5 
    #return np.abs(x[0]+x[1])*1.5 
    #return np.max([(x[0]+x[1])*2.5, 0.2])
    return np.max([(x[0]*x[1]+np.cos(x[2]*x[3]**2))*2.5, 0.2])
    #return 0.5 
    
def obj_func3(x):
    """
    conditional expectation
    """
    #return x[1]+2.*x[2] +2. +np.sin(2.*x[0])
    c = 0
    for i in range(4,7):
        #c *= x[i]*np.sin(i)
        c += x[i]*np.cos(i)
    #return x[1] + x[2]
    return c


#Y = np.random.normal(0,1.,N_total) + np.apply_along_axis(obj_func,1,X)
Y = np.zeros(N_total)
for i in range(N_total):
    if np.random.rand()<0.5:
        Y[i] = np.random.normal(obj_func(X[i]),np.sqrt(obj_func2(X[i])),1) 
    else:
        #Y[i] = np.random.normal(obj_func(X[i]),np.sqrt(obj_func2(X[i])),1) 
        # Y[i] = np.random.normal(-0.05*obj_func2(X[i]),1,1) 
        #Y[i] = np.random.normal(-1.5*obj_func2(X[i]),1,1) 
        #Y[i] = np.random.normal(obj_func3(X[i]),1,1) 
        
        Y[i] = np.random.normal(-1,1,1) 
        #Y[i] = np.random.normal(obj_func(X[i]),np.sqrt(obj_func2(X[i])),1) 

## Training

In [17]:
reg = WassersteinRandomForest(nodesize = 2,
                             bootstrap = False,
                             subsample = 500,
                             n_estimators = 200,
                             mtry = 1,
                             n_jobs = 1,
                             p = 2)
#reg = DecisionTree()
reg.fit(X[:N_train],Y[:N_train])




  0%|          | 0/200 [00:00<?, ?it/s]


  0%|          | 1/200 [00:05<18:31,  5.59s/it]


  1%|          | 2/200 [00:11<18:53,  5.72s/it]


  2%|▏         | 3/200 [00:17<18:42,  5.70s/it]

KeyboardInterrupt: 

## Conditional expectation estimation

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
predlist = reg.predict(X[N_train:])
print("Wasserstein RF")
print("R2:",r2_score(np.apply_along_axis(obj_func,1,X[N_train:]),predlist))
print("MSE:",
        np.sqrt(mean_squared_error(np.apply_along_axis(obj_func,1,X[N_train:]),predlist))
     )
#print("Memory usage:", process.memory_info().rss/1024/1024,"MB")

In [ ]:
# compare with classical Random Forests
from sklearn.ensemble import RandomForestRegressor
#from skgarden import MondrianForestRegressor

reg2 = RandomForestRegressor(n_estimators = 100,
                             min_samples_split = 2,
                             bootstrap = 1,
                             max_features = 1 
                            )
# reg2 = MondrianForestRegressor(n_estimators = 500,
#                             min_samples_split = 5,
#                             bootstrap = False)
reg2.fit(X[:N_train],Y[:N_train])

print("Classical RF")
print("R2:",r2_score(np.apply_along_axis(obj_func,1,X[N_train:]),reg2.predict(X[N_train:])))
print("MSE:",
        np.sqrt(mean_squared_error(np.apply_along_axis(obj_func,1,X[N_train:]),reg2.predict(X[N_train:])))
     )

## Conditional distribution estimation

In [ ]:
from seaborn import kdeplot

plt.figure(figsize = (20,15))
for IndexPlot in range(9):
    plt.subplot(int("33"+str(IndexPlot+1)))
    TestIndex = np.random.choice(range(N_total-N_train),1)[0]
    #TestIndex = np.random.choice(range(N_train),1)[0]
    #plt.hist(Y, density = True, color = "orange",alpha = 0.3, bins = 20, label="Y")
    kdeplot(Y, label="kde-Y", color = "darkorange")
    Y,W = reg.predict_distribution(X[-TestIndex])
    kdeplot(np.random.choice(a = Y,p = W,size = 1000), label="kde-pred", color = "black")
    plt.hist(Y,weights=W, bins = 20, color = "grey", density = True,alpha = 0.5, label="pred")
    #ref_sample = np.random.normal(obj_func(X[-TestIndex]),np.sqrt(obj_func2(X[-TestIndex])),2000)
    ref_sample = np.zeros(2000)
    for i in range(2000):
        if np.random.rand()<0.5:
            ref_sample[i] = np.random.normal(obj_func(X[-TestIndex]),np.sqrt(obj_func2(X[-TestIndex])),1)
        else:
            #ref_sample[i] = np.random.normal(-1.5*obj_func2(X[TestIndex]),1,1)
            # ref_sample[i] = np.random.normal(obj_func3(X[TestIndex]),1,1)

            ref_sample[i] = np.random.normal(-1,1,1)
            #ref_sample[i] = np.random.normal(obj_func(X[-TestIndex]),np.sqrt(obj_func2(X[-TestIndex])),1)

    plt.hist(ref_sample,
             density = True, color = "darkred",alpha = 0.3, bins = 20, label="ref")
    kdeplot(ref_sample, label = "kde-ref", color = "darkred")
    plt.grid(linestyle = "-.",color="lightgrey")
    plt.legend()
plt.show()


# Calculate average Wp distance with 100 points in test dataset

In [ ]:
from ot import wasserstein_1d
predlist = []
predlistY = []
ideallist = []
for i in range(100):
    Y_c,W_c = reg.predict_distribution(X[N_train+i])
    predlist +=[wasserstein_1d(p = 2,x_a = np.random.choice(Y_c,p=W_c,size = 100000),x_b =np.random.normal(obj_func(X[N_train+i]),np.sqrt(obj_func2(X[N_train+i])),100000))]
    #predlist +=[wasserstein_1d(p = 2,x_a = Y_c,a = W_c,x_b =np.random.normal(obj_func(X[N_train+i]),np.sqrt(obj_func2(X[N_train+i])),100000))]
    predlistY +=[wasserstein_1d(p = 2,x_a = Y[:N_train],x_b =np.random.normal(obj_func(X[N_train+i]),np.sqrt(obj_func2(X[N_train+i])),100000))]
    #ideallist += [wasserstein_1d(p = 2,x_a =np.random.normal(obj_func(X[N_train+i]),np.sqrt(obj_func2(X[N_train+i])),N_train),
                                   #x_b = np.random.normal(obj_func(X[N_train+i]),np.sqrt(obj_func2(X[N_train+i])),10000))] 
print("average Wp distance:", np.mean(predlist))
print("average Wp distance with Y (i.e., no estimation is made):", np.mean(predlistY))
#print("ideal average Wp distance", np.mean(ideallist))

In [ ]:
from ot import wasserstein_1d
predlist = []
predlistY = []
ideallist = []
for i in range(100):
    TestIndex = np.random.choice(range(N_total-N_train),1)[0]
    Y_c,W_c = reg.predict_distribution(X[-TestIndex])
    x_b_test = np.zeros(10000)
    for i in range(10000):
        if np.random.rand()<0.5:
            x_b_test[i] = np.random.normal(obj_func(X[-TestIndex]),np.sqrt(obj_func2(X[-TestIndex])),1)[0]
        else:
            x_b_test[i] = np.random.normal(-1,1,1)[0]
    predlist +=[wasserstein_1d(p = 2,x_a = np.random.choice(Y_c,p=W_c,size = 10000),x_b =x_b_test)]
print("average Wp distance:", np.mean(predlist))
#print("average Wp distance with Y (i.e., no estimation is made):", np.mean(predlistY))
#print("ideal average Wp distance", np.mean(ideallist))

In [ ]:


list_dist_rf = []
for j in tqdm(range(100)):
    TestIndex = np.random.choice(range(N_total-N_train),1)[0]
    listY = []
    for i in range(100):
        reg2 = RandomForestRegressor(n_estimators = 1,
                                     min_samples_split = 2,
                                     bootstrap = False,
                                     #max_samples = 200,
                                     max_features = 1 
                                    )
        # reg2 = MondrianForestRegressor(n_estimators = 500,
        #                             min_samples_split = 5,
        #                             bootstrap = False)
        reg2.fit(X[:N_train],Y[:N_train])
        listY += [reg2.predict(X[-TestIndex].reshape(1,X[-TestIndex].shape[0]))[0]]
    listY = np.array(listY)
    x_b_test = np.zeros(10000)
    for i in range(10000):
        if np.random.rand()<0.5:
            x_b_test[i] = np.random.normal(obj_func(X[-TestIndex]),np.sqrt(obj_func2(X[-TestIndex])),1)[0]
        else:
            x_b_test[i] = np.random.normal(-1,1,1)[0]
    list_dist_rf +=[wasserstein_1d(p = 2,x_a = listY,x_b = x_b_test)]

In [ ]:
np.mean(list_dist_rf)

In [ ]:
plt.figure(figsize = (15,6))
ref_sample = np.zeros(2000)
for i in range(2000):
    if np.random.rand()<0.5:
        ref_sample[i] = np.random.normal(obj_func(X[-TestIndex]),np.sqrt(obj_func2(X[-TestIndex])),1)
    else:
        #ref_sample[i] = np.random.normal(-1.5*obj_func2(X[TestIndex]),1,1)
        # ref_sample[i] = np.random.normal(obj_func3(X[TestIndex]),1,1)

        ref_sample[i] = np.random.normal(-1,1,1)
        #ref_sample[i] = np.random.normal(obj_func(X[-TestIndex]),np.sqrt(obj_func2(X[-TestIndex])),1)

kdeplot(listY, label="kde-rf")
Y_c,W_c = reg.predict_distribution(X[-TestIndex])
Y,W = reg.predict_distribution(X[-TestIndex])
#kdeplot(np.random.choice(a = Y,p = W,size = 1000), label="kde-pred", color = "black")
plt.subplot(121)
plt.hist(listY,bins = 30,density = True, label = "rf-pred")
plt.hist(ref_sample,density = True, color = "darkred",alpha = 0.3, bins = 30, label="ref")
#kdeplot(ref_sample, label = "kde-ref", color = "darkred")
plt.grid(linestyle="dotted")
plt.legend()
plt.subplot(122)
plt.hist(Y,weights=W, bins = 30, color = "grey", density = True,alpha = 1, label="pred")
plt.hist(ref_sample,density = True, color = "darkred",alpha = 0.3, bins = 30, label="ref")
plt.grid(linestyle="dotted")
plt.legend()